In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

col_names = ['Name', 'Risk', 'Return']

: 

In [ ]:
#cell to generate random data
rand_data = []
num_pts = 100

mean = 10 # average log normal return
returns_noise = 0.5 # log normal returns

for i in range(num_pts):
    risk = np.max(1 / (mean + np.random.normal()), 0)
    rand_data.append(["Disease " + str(i), risk, np.exp(np.log(mean) + returns_noise * np.random.normal())])
    
rand_data_df = pd.DataFrame(rand_data, columns = col_names)
rand_data_df.to_csv("./riskreturn.csv", index=False)

plt.figure()
plt.hist(rand_data_df['Risk'].tolist(), bins = 30)
plt.show()

plt.figure()
plt.hist(rand_data_df['Return'].tolist(), bins = 100)
plt.show()


: 

In [ ]:
data = pd.read_csv('./riskreturn.csv')

print(data)

: 

In [ ]:
n = len(data)
CovarRisk = np.array([[(data['Risk'][i]**2 if i == j else 0) for i in range(n)] for j in range(n)])
MeanReturns = data['Return'].tolist()

def MaximizeReturns(data, PortfolioSize): #optimize with no risk
    c = np.multiply(-1, data['Return'].tolist())
    A = np.ones([PortfolioSize, 1]).T
    b = [1]
    weights = opt.linprog(c, A_ub = A, b_ub = b, bounds = (0,1))
    return weights.x

def MinimizeReturns(data, size):
    def f(x):
        func = np.matmul(np.matmul(x, CovarRisk), x.T) 
        return func

    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    xinit=np.repeat(0.1, size)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    weights = opt.minimize (f, x0 = xinit,  bounds = bnds, \
                             constraints = cons, tol = 10**-3)
    
    return weights.x
    
def MeanVar(data, size, R):    
    def f(x, CovarRisk):
        return np.matmul(np.matmul(x, CovarRisk), x.T) 

    def constraintEq(x):
        AEq=np.ones(x.shape)
        bEq=1
        EqconstraintVal = np.matmul(AEq,x.T)-bEq 
        return EqconstraintVal
    
    def constraintIneq(x, MeanReturns, R):
        AIneq = np.array(MeanReturns)
        bIneq = R
        IneqconstraintVal = np.matmul(AIneq, x.T) - bIneq
        return IneqconstraintVal
    

    xinit=np.repeat(0.1, size)
    cons = ({'type': 'eq', 'fun':constraintEq},
            {'type':'ineq', 'fun':constraintIneq, 'args':(MeanReturns,R) })
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    res = opt.minimize (f, args = (CovarRisk), method ='trust-constr', x0 = xinit, bounds = bnds, constraints = cons)
    
    # if res.success == False:
    #     print("OPTIMIZE FAIL") 
    # else:
    #     print("OPTIMIZE SUCCESS") 
    return res.x

# print(MeanVar(data, 10, 10))

: 

In [ ]:
def ComputeReturns(data, weights):
    return np.matmul(np.array(data['Return'].tolist()), weights.T)

def ComputeRisk(weights, n):
    return np.sqrt(np.matmul(np.matmul(weights, CovarRisk), weights.T)) / np.sqrt(n)

n = len(data)
incr = 1
min = ComputeReturns(data, MinimizeReturns(data, n))
max = ComputeReturns(data, MaximizeReturns(data, n))

print(min)
print(max)

frontier_risk = []
frontier_return = []

R = min # target return
while R < max:
    res = MeanVar(data, n, R)
    frontier_risk.append(ComputeRisk(res, n))
    frontier_return.append(R)
    R += incr
    # print(R)

: 

In [ ]:
#plot results
plt.figure()
plt.plot(np.array(frontier_risk) * 100, frontier_return, color='orange')
plt.scatter(np.array(frontier_risk) * 100, frontier_return, s=10, color='orange')
plt.xlabel("Risk (%)")
plt.ylabel("Expected Return (%)")

: 

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
import pypfopt.objective_functions as objective_functions

expected_returns = data['Return']
cov_matrix = np.array([[(data['Risk'][i]**2 if i == j else 0) for i in range(n)] for j in range(n)])

ef = EfficientFrontier(expected_returns, cov_matrix)
ef.add_objective(objective_functions.L2_reg)
weights = ef.efficient_risk(0.04)
n = len(data)
incr = 0.001
min = 0.032
max = 0.1

print(min)
print(max)

frontier_risk = []
frontier_return = []
frontier_weights = []
R = min # risk tolerance
while R < max:
    res = ef.efficient_risk(R)
    res = np.array([i[1] for i in res.items()])
    R += incr
    frontier_risk.append(R)
    frontier_return.append(ComputeReturns(data, res))
    frontier_weights.append(res)
    
plt.figure()
plt.scatter(frontier_risk, frontier_return)
plt.xlabel("Risk (%)")
plt.ylabel("Expected Return (%)")

output_data = pd.DataFrame(zip(frontier_risk,frontier_return, frontier_weights)).to_csv("./efficient_frontier.csv")

: 